# ELEC474 Lab2 20119641
please check the console down below for more info when running the program 

## Import

In [1]:
import cv2 as cv
import numpy as np
import random
import math

## Initialzation Param

In [2]:
#I tried two implement them in the same page, but kind of hard and will make code bulky 
#So sorry for required need uncomment this again 
# imgName = "baboon.jpg"
imgName = "SAFlag.jpg" #comment this if u need to test baboon.jpg 
windowName = "Lab2_20119641"
trackbarName = "K value"
trackbarValRange = [0, 30]  #the range for the k cluster 
maxRoundAllow = 15          #the maximum round for redoing k-mean based on changing mean value
centoridApproxThreshold = 1 #the threshold for mean of BGR channel diff between mean 
                            #of new centroid and old centroid


## Callback Functions

In [3]:
def TrackbarCallback(*arg):
    global kVal
    kVal = cv.getTrackbarPos(trackbarName, windowName)  #assign kval from trackbar


## Aid functions


In [4]:
def ColorValDiff(point1, point2):       #calculating the mean BGR value of two given pixels 
    try:                                #try catch section can be removed 
        val = CalDiff(point1[0], point2[0])+CalDiff(point1[1], point2[1])+CalDiff(point1[2], point2[2])  
        return val
    except:
        print("centoid : ", point1)
        print("compare point: ", point2)

In [5]:
def CalDiff(x, y):                      #calculate diff based on below equation, time consume type, can be imporve
    return math.sqrt(math.pow(x-y, 2))

In [6]:
def ImageRestore(imgSize):  #Try to restore the color of image in the window when every time change kVal  
    global imgOrig,windowName,img
    for x in range(imgSize[0]):
        for y in range(imgSize[1]):
            img[x][y] = imgOrig[x][y]
    cv.imshow(windowName, img) #don't know why is not working LOL

## k-mean cluster algorithm implemetation

In [7]:
def initCentroids(img, imgSize, kVal):
    centroids = np.zeros((kVal,img.shape[2]))                   
    for i in range(kVal):                               #randomly pick up centriod based on kVal
        randX = random.randint(0, imgSize[0]-1)
        randY = random.randint(0,imgSize[1]-1)
        centroids[i, :] = np.copy(img[randX][randY])
    # print("inital centroid point:\n", centroids)      #uncomment if needed 
    return centroids                                    #retrun in each row [#'s represent RGB value]

In [8]:
def Kmeans(img, imgSize, kVal):
    
    global firstTest
    if not firstTest:                                #not important, reusing image only
        print(30*'*',"restore image",30*'*')
        ImageRestore(imgSize)
    if kVal == 0:
        print("Please choice a k val bigger than 0")
        return
    # the first colum, [x,y,0], stores which cluster this sample belongs to
    # the second colum , [x,y,1], stores the diff between sample pixel and certorid 
    clusterAssign = np.zeros((imgSize[0],imgSize[1], 2))
    isCountiue = True

    ##init cluster centoird 
    centroid = initCentroids(img, imgSize,kVal)

    #performance cal init
    print (40*'=')
    print("Start k-mean cluster under k value of ", kVal)
    time_All_start = cv.getTickCount()
    countRound = 0
    
    #k means algorithm 
    while isCountiue and countRound < maxRoundAllow:

        countRound += 1
        print(10*'-',"> start round ", countRound)
        time_Round_Start = cv.getTickCount()

        isCountiue = False
        #walk through all the pixel and compare with all the random centroid
        for x in range(imgSize[0]):
            for y in range(imgSize[1]):
                minDiff = 255*255*255 #just get a random large val 
                minIdx = 0
                for i in range(kVal):
                    tempDiff = ColorValDiff(centroid[i], img[x][y]) #get the diff value for this point when centroid 
                    if tempDiff < minDiff and CalDiff(tempDiff, minDiff) > centoridApproxThreshold :
                        minDiff = tempDiff
                        minIdx = i
                clusterAssign[x,y,:] = [minIdx,minDiff] #store the cluster type and diff value
        
        #initalization 
        meanVal = []
        diffVal = []
        badCount = 0
        
        for j in range(kVal):           #Calculate the sum of mean for all point in a cluster
            locator = np.nonzero(clusterAssign[:,:,0] == j)
            clusterPVal = img[locator]
            meanVal.append(np.mean(clusterPVal, axis=0))          
        for i in range(len(centroid)):  #compare with threshold to see whether should redo clustering cal
            tempDiff = ColorValDiff(centroid[i], meanVal[i])
            diffVal.append(tempDiff)
            if  tempDiff >= centoridApproxThreshold:
                badCount += 1           # redundant use due to debug 
        if badCount == 0:               # End k-mean looping if all pixel satisfied with thershold
            print(10*'！')
            isCountiue = False
        else:                           #else keep looping and update centroid with meanVal forall k cluster
            isCountiue = True
            centroid = meanVal
            
        #performance
        time_round_end = cv.getTickCount()
        time_round_cal = (time_round_end - time_Round_Start)/cv.getTickFrequency()
#         print("k centroid mean cluster:\n", centroid)         #uncomment if needed， debug useage
        print("Average of difference for all k", np.mean(diffVal))
        print("End round", countRound, "with in", time_round_cal, "sec <",10*'-','\n')

    #performance
    time_All_end = cv.getTickCount()
    time_All_cal = (time_All_end - time_All_start) / cv.getTickFrequency()
    print("!!!!!!!CONGRATULATION!!!!!!!!")
    print("Cluster finished in", time_All_cal, "sec with", countRound, "Rounds")
    print (40*'=')

    #perform ColorQuantization
    print(15*'>', "Start ColorQuantization", 15*'>')
    time_All_start = cv.getTickCount()
    ColorQuantization(centroid,clusterAssign,img,imgSize)
    time_All_end = cv.getTickCount()
    time_All_cal = (time_All_end - time_All_start) / cv.getTickFrequency()
    print(10*'<',"Color Quantization finished in ", time_All_cal, "sec",10*'<')

## Color Quantization

In [9]:
def ColorQuantization(centroid, clusterAssign, img, imgSize):
    global windowName, firstTest
    for x in range(imgSize[0]):
        for y in range(imgSize[1]):
            idx = int(clusterAssign[x,y,0])
            img[x,y] = centroid[idx]
    cv.imshow(windowName, img)
    firstTest = False


## Main Function

In [10]:
img = cv.imread(imgName)
imgOrig = np.copy(img)
imgSize = [img.shape[0], img.shape[1]]
kVal = 0 #num of centers 
firstTest = True

cv.namedWindow(windowName, cv.WINDOW_NORMAL)
cv.resizeWindow(windowName,imgSize[1], imgSize[0])
cv.createTrackbar(trackbarName, windowName, trackbarValRange[0],trackbarValRange[1], TrackbarCallback)
print("Press ENTER to start K mean cluster after u pick a k Val")
print("Press ESC to exit the program")
cv.imshow(windowName, img)
while True:
    key = cv.waitKey(1)
    if key == 27: #press ESC to exit the program
        break
    if key == 13: #press ENTER to start k-mean clustering
        Kmeans(img, imgSize, kVal)
cv.destroyAllWindows()

Press ENTER to start K mean cluster after u pick a k Val
Press ESC to exit the program
Please choice a k val bigger than 0
Start k-mean cluster under k value of  6
---------- > start round  1
Average of difference for all k 33.521774109270304
End round 1 with in 4.2618754 sec < ---------- 

---------- > start round  2
Average of difference for all k 7.337853895218726
End round 2 with in 4.1296924 sec < ---------- 

---------- > start round  3
Average of difference for all k 3.8166218531754073
End round 3 with in 4.1164417 sec < ---------- 

---------- > start round  4
Average of difference for all k 3.6924691610214784
End round 4 with in 4.1124888 sec < ---------- 

---------- > start round  5
Average of difference for all k 4.088854901966182
End round 5 with in 4.1099577 sec < ---------- 

---------- > start round  6
Average of difference for all k 4.0931650490324705
End round 6 with in 4.0973274 sec < ---------- 

---------- > start round  7
Average of difference for all k 4.84268941